In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import cv2 as cv
import time
import os
import segmentation_models_pytorch.utils as smp_utils 
from networks import resnet50
from complexnn import ComplexBatchNorm2d


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\gadge\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

In [4]:
VOC_COLORMAP = [
    [0, 0, 0],
    [128, 0, 0],
    [0, 128, 0],
    [128, 128, 0],
    [0, 0, 128],
    [128, 0, 128],
    [0, 128, 128],
    [128, 128, 128],
    [64, 0, 0],
    [192, 0, 0],
    [64, 128, 0],
    [192, 128, 0],
    [64, 0, 128],
    [192, 0, 128],
    [64, 128, 128],
    [192, 128, 128],
    [0, 64, 0],
    [128, 64, 0],
    [0, 192, 0],
    [128, 192, 0],
    [0, 64, 128],
]

In [5]:
class VocDataset(Dataset):
    def __init__(self, dir, color_map):
        self.root = os.path.join(dir, 'VOC2012')
        self.target_dir = os.path.join(self.root, 'SegmentationClass')
        self.images_dir = os.path.join(self.root, 'JPEGImages')
        file_list = os.path.join(self.root, 'ImageSets/Segmentation/trainval.txt')
        self.files = [line.rstrip() for line in tuple(open(file_list, "r"))]
        self.color_map = color_map

        # Define the transformations for the images and the masks
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            ToHSV(),
            ToComplex()
        ])

    def convert_to_segmentation_mask(self, mask):
        # This function converts color channels of segmentation masks to number of classes (21 in this case)
        height, width = mask.shape[:2]
        segmentation_mask = torch.zeros((height, width, len(self.color_map)), dtype=torch.float32)
        for label_index, label in enumerate(self.color_map):
            segmentation_mask[:, :, label_index] = (mask == torch.tensor(label, dtype=torch.uint8).unsqueeze(0).unsqueeze(0)).all(dim=-1).float()
        return segmentation_mask

    def __getitem__(self, index):
        image_id = self.files[index]
        image_path = os.path.join(self.images_dir, f"{image_id}.jpg")
        label_path = os.path.join(self.target_dir, f"{image_id}.png")

        # Open the image and label using PIL
        image = Image.open(image_path).convert('RGB')
        # label = Image.open(label_path).convert('RGB')

        # Apply transformations
        image = self.transform(image)
        # label = self.transform(label)

        # Convert labels to segmentation masks
        label = self.convert_to_segmentage_mask(np.array(label.permute(1, 2, 0)))
        return image, label



  
    def __len__(self):
        return len(self.files)

data=VocDataset('voc_dataset',VOC_COLORMAP)

batch_size = 10

train_set,val_set=torch.utils.data.random_split(data,[int(len(data)*0.9),round(len(data)*0.1)+1])
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False)

print('Train Size   : ', len(train_set))
print('Val Size     : ', len(val_set))

FileNotFoundError: [Errno 2] No such file or directory: 'voc_dataset\\VOC2012\\ImageSets/Segmentation/trainval.txt'